In [ ]:

!pip install ultralytics deep-sort-realtime opencv-python matplotlib --quiet


In [ ]:

# Download MOT20-02 sequence (images only)
!wget -O /content/MOT20-02.zip "https://motchallenge.net/data/MOT20/MOT20-02.zip"

# Unzip dataset
!unzip -q /content/MOT20-02.zip -d /content/MOT20

# Path to frames
FRAMES_PATH = "/content/MOT20/train/MOT20-02/img1"


In [ ]:

import cv2
import os
import glob
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import matplotlib.pyplot as plt
import pandas as pd

# Load YOLOv8 person detection model
model = YOLO('yolov8n.pt')

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=30)


In [ ]:

# Parameters
CROWD_THRESHOLD = 10
output_video_path = "/content/crowd_output.mp4"

# Get all frames sorted
frame_files = sorted(glob.glob(os.path.join(FRAMES_PATH, "*.jpg")))

# Prepare video writer
sample_frame = cv2.imread(frame_files[0])
h, w, _ = sample_frame.shape
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, 30, (w, h))

# Data storage
frame_counts = []

for idx, frame_path in enumerate(frame_files):
    frame = cv2.imread(frame_path)
    results = model(frame, verbose=False)
    
    # Filter detections for 'person' class only
    detections = []
    for r in results:
        for box in r.boxes:
            cls = int(box.cls[0])
            if model.names[cls] == "person":
                x1, y1, x2, y2 = box.xyxy[0]
                conf = float(box.conf[0])
                detections.append(((x1, y1, x2-x1, y2-y1), conf, cls))
    
    # Update tracker
    tracks = tracker.update_tracks(detections, frame=frame)
    
    # Draw boxes
    count_people = 0
    for t in tracks:
        if not t.is_confirmed():
            continue
        count_people += 1
        ltrb = t.to_ltrb()
        cv2.rectangle(frame, (int(ltrb[0]), int(ltrb[1])), (int(ltrb[2]), int(ltrb[3])), (0,255,0), 2)
    
    # Add alert if crowd exceeds threshold
    if count_people > CROWD_THRESHOLD:
        cv2.putText(frame, "OVER CROWD ALERT!", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)
    
    # Save count
    frame_counts.append({"frame": idx, "count": count_people})
    
    # Write frame
    out.write(frame)

out.release()

# Save counts to CSV
df_counts = pd.DataFrame(frame_counts)
df_counts.to_csv("/content/crowd_counts.csv", index=False)


In [ ]:

# Plot crowd counts over time
df = pd.read_csv("/content/crowd_counts.csv")
plt.figure(figsize=(12,6))
plt.plot(df["frame"], df["count"], label="People Count")
plt.axhline(y=CROWD_THRESHOLD, color="r", linestyle="--", label="Crowd Threshold")
plt.xlabel("Frame")
plt.ylabel("Number of People")
plt.title("Crowd Count Over Time (MOT20-02)")
plt.legend()
plt.show()
